# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [18]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json
# Import API key
from api_keys import geoapify_key

In [27]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/City_data.csv")
# Display sample data
city_data_df.rename(columns={'Unnamed: 0' : 'City_ID'}, inplace = True)
city_data_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ushuaia,-54.8000,-68.3000,0.81,93,100,10.29,AR,1686528754
1,1,qamdo,31.1667,97.2333,2.83,85,100,1.83,CN,1686528754
2,2,parras de la fuente,25.4167,-102.1833,33.32,12,2,5.61,MX,1686528754
3,3,bitkine,11.9801,18.2138,28.52,60,70,2.36,TD,1686528754
4,4,susuman,62.7833,148.1667,22.39,36,49,6.08,RU,1686528754


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [20]:
%%capture --no-display

city_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City",
    #hover_cols = ["Address", "Bank Count"])
    
)


# Display the map
city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [30]:
# Narrow down cities that fit criteria and drop any results with null values
# A max temperature lower than 27 degrees but higher than 21
# Wind speed less than 4.5 m/s
# Zero cloudiness
#city_data_df.loc([city_data_df['Lat']] >= 0)
city_data_df = city_data_df.loc[(city_data_df['Max Temp'] >= 21) & (city_data_df['Wind Speed'] < 4.5)
                                 & (city_data_df['Max Temp'] < 27) & (city_data_df['Cloudiness'] == 0)
                               & (city_data_df['Humidity'] <60)]                        
# Drop any rows with null values
city_data_df.dropna()
# Display sample data
city_data_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
9,9,mizdah,31.4451,12.9801,22.91,57,0,4.44,LY,1686528755
149,149,doaba,33.4244,70.7364,26.71,38,0,1.21,PK,1686528770
196,196,pierre,44.3683,-100.3510,23.51,43,0,1.79,US,1686528774
259,259,colonia,50.9333,6.9500,21.08,51,0,2.57,DE,1686528781
288,288,smithers,54.7804,-127.1743,25.18,22,0,3.60,CA,1686528721
325,325,monaco,43.7314,7.4190,22.88,56,0,1.03,MC,1686528788
357,357,bismarck,46.8083,-100.7837,23.26,39,0,3.60,US,1686528509
481,481,yelizovo,53.1873,158.3805,21.08,35,0,2.00,RU,1686528807
517,517,kandahar,31.0000,65.7500,22.75,21,0,2.86,AF,1686528810
518,518,detroit lakes,46.8172,-95.8453,22.03,30,0,4.12,US,1686528779


### Step 3: Create a new DataFrame called `hotel_df`.

In [32]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = city_data_df.drop(columns=['City_ID','Max Temp','Cloudiness','Wind Speed','Date'])
hotel_df['Hotel Name'] = ''
hotel_df

,City,Lat,Lng,Humidity,Country,Hotel Name
9,mizdah,31.4451,12.9801,57,LY,
149,doaba,33.4244,70.7364,38,PK,
196,pierre,44.3683,-100.3510,43,US,
259,colonia,50.9333,6.9500,51,DE,
288,smithers,54.7804,-127.1743,22,CA,
325,monaco,43.7314,7.4190,56,MC,
357,bismarck,46.8083,-100.7837,39,US,
481,yelizovo,53.1873,158.3805,35,RU,
517,kandahar,31.0000,65.7500,21,AF,
518,detroit lakes,46.8172,-95.8453,30,US,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [33]:

# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
limit = 20
params = {
    "apiKey":geoapify_key,
    "categories":categories,
    "limit":limit
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params).json()
    
    # Convert the API response to JSON format
    #name_address = # YOUR CODE HERE
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
mizdah - nearest hotel: No hotel found
doaba - nearest hotel: No hotel found
pierre - nearest hotel: Quality Inn
colonia - nearest hotel: Wasserturm Hotel Cologne
smithers - nearest hotel: Sunshine Inn Hotel
monaco - nearest hotel: Ambassador
bismarck - nearest hotel: Radisson Hotel Bismarck
yelizovo - nearest hotel: Николина Усадьба
kandahar - nearest hotel: No hotel found
detroit lakes - nearest hotel: Graystone Hotel
kolonia - nearest hotel: Wasserturm Hotel Cologne


,City,Lat,Lng,Humidity,Country,Hotel Name
9,mizdah,31.4451,12.9801,57,LY,No hotel found
149,doaba,33.4244,70.7364,38,PK,No hotel found
196,pierre,44.3683,-100.3510,43,US,Quality Inn
259,colonia,50.9333,6.9500,51,DE,Wasserturm Hotel Cologne
288,smithers,54.7804,-127.1743,22,CA,Sunshine Inn Hotel
325,monaco,43.7314,7.4190,56,MC,Ambassador
357,bismarck,46.8083,-100.7837,39,US,Radisson Hotel Bismarck
481,yelizovo,53.1873,158.3805,35,RU,Николина Усадьба
517,kandahar,31.0000,65.7500,21,AF,No hotel found
518,detroit lakes,46.8172,-95.8453,30,US,Graystone Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [35]:
%%capture --no-display

hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name", "Country","Humidity"]
    
)

hotel_map


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)